# Not a scaling law!

We will play with the [Transformers](https://huggingface.co/docs/transformers/en/index) and [Datasets](https://huggingface.co/docs/datasets/en/index) librairies of Hugging Face.

The question is: given a budget of compute, what is the impact of data scarcity?

To simplify, we will add some constraints:
- the number of parameters is fixed: you will fine-tune the distilGPT2 model.
- we fix the maximum sentence length at 64 tokens and assume the compute budget allows you to pass at most 100 times one sentence forward and backward through the model.

In [1]:
import time
import datasets
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling

The code below loads the corpus

In [2]:
t = GPT2TokenizerFast.from_pretrained('distilgpt2')
t.pad_token = t.eos_token
dc = DataCollatorForLanguageModeling(tokenizer=t, mlm=False)
d0 = datasets.load_dataset("wikitext","wikitext-2-v1")
dval = d0['validation']
dtrain = d0['train']

/home/danube/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/6.07M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/618k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

The code below constructs a training dataset and a validation dataset

In [3]:
slen = 64
def tokenize(element):
    outputs = t(element["text"], truncation=True, max_length=slen, return_overflowing_tokens=True, return_length=True)
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == slen: input_batch.append(input_ids)
    return {"input_ids": input_batch}
dtrain = dtrain.map(tokenize, batched=True, remove_columns=dtrain.column_names)
dval = dval.map(tokenize, batched=True, remove_columns=dval.column_names)
print("training data",d0)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

training data DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [4]:
dval = dval.select([i for i in range(10)])
print("validation data",dval)

validation data Dataset({
    features: ['input_ids'],
    num_rows: 10
})


In [9]:
! pip install transformers[torch]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


Here is an example of code for finetuning with the [`Trainer` class](https://huggingface.co/docs/transformers/en/main_classes/trainer).

In [10]:
d = dtrain.select([i for i in range(3)])
model = GPT2LMHeadModel.from_pretrained('distilgpt2')
trargs = TrainingArguments(".", do_train=True, num_train_epochs=5, per_device_train_batch_size=1, logging_steps=1, learning_rate=0.0001,
            per_device_eval_batch_size=1, eval_steps=1, report_to="none")
tr = Trainer(model=model, args=trargs, train_dataset=d, eval_dataset=dval, processing_class=t, data_collator=dc)
tr.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

Given the constraints above, what experiment should you do in order to see the impact of data scarcity? 